In [ ]:
library(yaml)
library(tidyr)
library(data.table)
library(ggplot2)
library(cowplot)

theme_set(theme_cowplot())
options(repr.plot.width=6, repr.plot.height=5)

In [ ]:
config <- yaml.load_file("../config_small.yaml")
results <- fread(file.path(config$ROOT, "cc_variance.csv"))
results <- separate(results, col = 'setting', sep = '/', c('scenario', 'analysis', 'scaling', 'feature_selection', 'method'))
results <- separate(results, col = 'method', sep = '_', c('method', 'output_type'))

In [ ]:
head(results)

In [ ]:
ggplot(results, aes(method, score, col=method)) +
    geom_boxplot() +
    facet_wrap(.~feature_selection) +
    labs(title="Cell cycle conservation score") +
    scale_color_brewer(palette = 'Dark2') +
    theme(legend.position = 'none',
          axis.text.x = element_text(angle=45, hjust=1))

In [ ]:
ggplot(results, aes(before, after, col=scaling)) +
    geom_point() + geom_abline() +
    facet_wrap(.~feature_selection) +
    labs(title="Cell cycle variance contribution before/after integration") +
    scale_color_brewer(palette = 'Set1') +
    theme(legend.position = 'top')

In [ ]:
ggplot(results[after>before], aes(before, after, col=scaling)) +
    geom_point() + geom_abline() +
    facet_wrap(.~feature_selection) +
    scale_color_brewer(palette = 'Set1') +
    theme(legend.position = 'top')

In [ ]:
# Compare cc score with recomputation from variance contributions
results[, lfc := (after/before)]
results[, abs := 1 - (abs(before-after)/before)]
results[, non_abs := ((before - (before-after))/before)]

In [ ]:
head(results)

In [ ]:
ggplot(results, aes(score, lfc, col=feature_selection)) +
    geom_point() + geom_abline() +
    facet_wrap(.~method) +
    labs(x="cell cycle score", y = "variance ratio: after/before",
         title = "Score comparison") + 
    scale_color_brewer(palette = 'Dark2') +
    theme(legend.position = 'top')